In [100]:
import pymysql
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.utils import shuffle
import datetime
import time
import warnings
warnings.filterwarnings("ignore")
import sys
sys.path.append(r'D:\jupyter files\waiting_time_project\my_tools')
import tools_for_os.for_df as ml_df

In [107]:
# 把字符串转成datetime
def str_to_Datetime(st):
    dt = datetime.datetime.strptime(st, "%Y-%m-%d %H:%M:%S")
    return dt

# 把datetime 转成字符串
def Datetime_to_str(d):
    str_date = d.strftime('%Y-%m-%d %H:%M:%S')
    return str_date

# 计算时间差
def cal_time_inv(before, now):
    before = str_to_Datetime(before)
    now = str_to_Datetime(now)
    if now > before:
        inv = (now - before).seconds
        return inv
    else:
        return -1

=========================================================

#### 这一部分的数据与处理和治疗时长预测的预处理相同

都需要将同一个appointment 里面的特征合并，同时将分类变量进行编码

In [13]:
data_path = 'D:\\jupyter files\\data_waiting_time_project\\preprocess_data\\'

data_part1 = pd.read_csv(data_path + 'data_part1.csv', index_col = 0)
data_part2 = pd.read_csv(data_path + 'data_part2.csv', index_col = 0)

DATA = pd.merge(data_part1, data_part2, on = ['PatientSerNum', 'date'], how = 'inner')
DATA.sort_values(by = ['PatientSerNum', 'AppointmentSerNum', 'ScheduledStartTime', 'FractionNumber'], inplace = True)
print(f'The shape of data is {DATA.shape}')

C:\Users\WENDY\Anaconda3\lib\site-packages\numpy\lib\arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


The shape of data is (1788434, 35)


In [14]:
feature_num = ['Scheduled_duration', 'Actual_duration',
               'age', 'TreatmentTime_total', 'ImagesTaken_total',
               'MU_total', 'MUCoeff_total', 'Interval_scheduled']

# RadiationId
feature_cate = ['dxt_AliasName', 'Sex', 'AliasSerNum',
                'month', 'week', 'hour', 'DoctorSerNum', 
                'TreatmentOrientation', 'FractionNumber',
                'UserName', 'CourseId', 'ResourceSerNum']

In [15]:
# 因为需要进行onehot encoding，所以在拼接数据之前，先进行数据格式的处理
for col in feature_cate:
    try:
        data_part1[col].fillna('Unknown', inplace = True)
        data_part1[col] = data_part1[col].astype(str)
    except:
        pass
    
    try:
        data_part2[col].fillna('Unknown', inplace = True)
        data_part2[col] = data_part2[col].astype(str)
    except:
        pass

for col in feature_num:
    try:
        data_part1.fillna(0, inplace = True)
    except:
        pass
    try:
        data_part2.fillna(0, inplace = True)
    except:
        pass
    
# data_num = log1p(data_num)

In [16]:
# one hot encoder
label_encoder_dxt_AliasName = preprocessing.OneHotEncoder(handle_unknown = 'ignore', sparse = False)
label_encoder_dxt_AliasName.fit(data_part1.dxt_AliasName.values.reshape(-1, 1))

label_encoder_Sex = preprocessing.OneHotEncoder(handle_unknown = 'ignore', sparse = False)
label_encoder_Sex.fit(data_part1.Sex.values.reshape(-1, 1))

label_encoder_AliasSerNum = preprocessing.OneHotEncoder(handle_unknown = 'ignore', sparse = False)
label_encoder_AliasSerNum.fit(data_part1.AliasSerNum.values.reshape(-1, 1))

label_encoder_month = preprocessing.OneHotEncoder(handle_unknown = 'ignore', sparse = False)
label_encoder_month.fit(data_part1.month.values.reshape(-1, 1))

label_encoder_week = preprocessing.OneHotEncoder(handle_unknown = 'ignore', sparse = False)
label_encoder_week.fit(data_part1.week.values.reshape(-1, 1))

label_encoder_hour = preprocessing.OneHotEncoder(handle_unknown = 'ignore', sparse = False)
label_encoder_hour.fit(data_part1.hour.values.reshape(-1, 1))

label_encoder_DoctorSerNum = preprocessing.OneHotEncoder(handle_unknown = 'ignore', sparse = False)
label_encoder_DoctorSerNum.fit(data_part1.DoctorSerNum.values.reshape(-1, 1))

label_encoder_TreatmentOrientation = preprocessing.OneHotEncoder(handle_unknown = 'ignore', sparse = False)
label_encoder_TreatmentOrientation.fit(data_part1.TreatmentOrientation.values.reshape(-1, 1))

label_encoder_FractionNumber = preprocessing.OneHotEncoder(handle_unknown = 'ignore', sparse = False)
label_encoder_FractionNumber.fit(data_part2.FractionNumber.values.reshape(-1, 1))

label_encoder_UserName = preprocessing.OneHotEncoder(handle_unknown = 'ignore', sparse = False)
label_encoder_UserName.fit(data_part2.UserName.values.reshape(-1, 1))

label_encoder_CourseId = preprocessing.OneHotEncoder(handle_unknown = 'ignore', sparse = False)
label_encoder_CourseId.fit(data_part2.CourseId.values.reshape(-1, 1))

label_encoder_ResourceSerNum = preprocessing.OneHotEncoder(handle_unknown = 'ignore', sparse = False)
label_encoder_ResourceSerNum.fit(data_part2.ResourceSerNum.values.reshape(-1, 1))

OneHotEncoder(categorical_features=None, categories=None, drop=None,
              dtype=<class 'numpy.float64'>, handle_unknown='ignore',
              n_values=None, sparse=False)

In [18]:
def get_list(x):
    x = list(x)
    if len(x) == 1:
        return x[0]
    else:
        return x

In [19]:
new_appt = pd.DataFrame({})

print('Start cateorical features')
print('\nStart PatientSerNum')
new_appt['PatientSerNum'] = data_part1.groupby('AppointmentSerNum').PatientSerNum.apply(set)
new_appt['PatientSerNum'] = new_appt['PatientSerNum'].apply(lambda x: get_list(x))

print('Start Sex')
new_appt['Sex'] = data_part1.groupby('AppointmentSerNum').Sex.apply(set)
new_appt['Sex'] = new_appt['Sex'].apply(lambda x: get_list(x))

print('Start DoctorSerNum')
new_appt['DoctorSerNum'] = data_part1.groupby('AppointmentSerNum').DoctorSerNum.apply(set)
new_appt['DoctorSerNum'] = new_appt['DoctorSerNum'].apply(lambda x: get_list(x))

print('Start date')
new_appt['date'] = data_part1.groupby('AppointmentSerNum').date.apply(set)
new_appt['date'] = new_appt['date'].apply(lambda x: get_list(x))

print('Start ScheduledStartTime')
new_appt['ScheduledStartTime'] = data_part1.groupby('AppointmentSerNum').ScheduledStartTime.apply(set)
new_appt['ScheduledStartTime'] = new_appt['ScheduledStartTime'].apply(lambda x: get_list(x))

print('Start ScheduledEndTime')
new_appt['ScheduledEndTime'] = data_part1.groupby('AppointmentSerNum').ScheduledEndTime.apply(set)
new_appt['ScheduledEndTime'] = new_appt['ScheduledEndTime'].apply(lambda x: get_list(x))

print('Start ActualStartDate')
new_appt['ActualStartDate'] = data_part1.groupby('AppointmentSerNum').ActualStartDate.apply(set)
new_appt['ActualStartDate'] = new_appt['ActualStartDate'].apply(lambda x: get_list(x))

print('Start ActualEndDate')
new_appt['ActualEndDate'] = data_part1.groupby('AppointmentSerNum').ActualEndDate.apply(set)
new_appt['ActualEndDate'] = new_appt['ActualEndDate'].apply(lambda x: get_list(x))

print('Start dxt_AliasName')
new_appt['dxt_AliasName'] = data_part1.groupby('AppointmentSerNum').dxt_AliasName.apply(set)
new_appt['dxt_AliasName'] = new_appt['dxt_AliasName'].apply(lambda x: get_list(x))

print('Start AliasSerNum')
new_appt['AliasSerNum'] = data_part1.groupby('AppointmentSerNum').AliasSerNum.apply(set)
new_appt['AliasSerNum'] = new_appt['AliasSerNum'].apply(lambda x: get_list(x))

print('Start CourseSerNum')
new_appt['CourseSerNum'] = data_part1.groupby('AppointmentSerNum').CourseSerNum.apply(set)
new_appt['CourseSerNum'] = new_appt['CourseSerNum'].apply(lambda x: get_list(x))

print('Start PlanSerNum')
new_appt['PlanSerNum'] = data_part1.groupby('AppointmentSerNum').PlanSerNum.apply(set)
new_appt['PlanSerNum'] = new_appt['PlanSerNum'].apply(lambda x: get_list(x))

print('Start TreatmentOrientation')
new_appt['TreatmentOrientation'] = data_part1.groupby('AppointmentSerNum').TreatmentOrientation.apply(set)
new_appt['TreatmentOrientation'] = new_appt['TreatmentOrientation'].apply(lambda x: get_list(x))

print('Start month')
new_appt['month'] = data_part1.groupby('AppointmentSerNum').month.apply(set)
new_appt['month'] = new_appt['month'].apply(lambda x: get_list(x))

print('Start week')
new_appt['week'] = data_part1.groupby('AppointmentSerNum').week.apply(set)
new_appt['week'] = new_appt['week'].apply(lambda x: get_list(x))

print('Start hour')
new_appt['hour'] = data_part1.groupby('AppointmentSerNum').hour.apply(set)
new_appt['hour'] = new_appt['hour'].apply(lambda x: get_list(x))

print('Start AppointmentSerNum')
new_appt['AppointmentSerNum'] = new_appt.index.tolist()


print('Start numberical features')
print('\nStart age')
new_appt['age'] = data_part1.groupby('AppointmentSerNum').age.mean()

print('Start Scheduled_duration')
new_appt['Scheduled_duration'] = data_part1.groupby('AppointmentSerNum').Scheduled_duration.mean()

print('Start Actual_duration')
new_appt['Actual_duration'] = data_part1.groupby('AppointmentSerNum').Actual_duration.mean()

Start cateorical features

Start PatientSerNum
Start Sex
Start DoctorSerNum
Start date
Start ScheduledStartTime
Start ScheduledEndTime
Start ActualStartDate
Start ActualEndDate
Start dxt_AliasName
Start AliasSerNum
Start CourseSerNum
Start PlanSerNum
Start TreatmentOrientation
Start month
Start week
Start hour
Start AppointmentSerNum
Start numberical features

Start age
Start Scheduled_duration
Start Actual_duration


In [20]:
new_treat = pd.DataFrame({})

print('Start categorical features')
print('\nStart FractionNumber')
new_treat['FractionNumber'] = data_part2.groupby(['PatientSerNum', 'date']).FractionNumber.apply(set)
new_treat['FractionNumber'] = new_treat['FractionNumber'].apply(lambda x: get_list(x))

print('Start UserName')
new_treat['UserName'] = data_part2.groupby(['PatientSerNum', 'date']).UserName.apply(set)
new_treat['UserName'] = new_treat['UserName'].apply(lambda x: get_list(x))

print('Start RadiationSerNum')
new_treat['RadiationSerNum'] = data_part2.groupby(['PatientSerNum', 'date']).RadiationSerNum.apply(set)
new_treat['RadiationSerNum'] = new_treat['RadiationSerNum'].apply(lambda x: get_list(x))

print('Start RadiationId')
new_treat['RadiationId'] = data_part2.groupby(['PatientSerNum', 'date']).RadiationId.apply(set)
new_treat['RadiationId'] = new_treat['RadiationId'].apply(lambda x: get_list(x))

# print('Start ResourceSerNum')
# new_treat['ResourceSerNum'] = data_part2.groupby(['PatientSerNum', 'date']).ResourceSerNum.apply(set)
# new_treat['ResourceSerNum'] = new_treat['ResourceSerNum'].apply(lambda x: get_list(x))

print('Start CourseId')
new_treat['CourseId'] = data_part2.groupby(['PatientSerNum', 'date']).CourseId.apply(set)
new_treat['CourseId'] = new_treat['CourseId'].apply(lambda x: get_list(x))

print('Start PatientSerNum')
new_treat['PatientSerNum'] = new_treat.index.get_level_values(level = 0).tolist()

print('Start date')
new_treat['date'] = new_treat.index.get_level_values(level = 1).tolist()


print('Start numberical features')
print('\nStart ImagesTaken_total')
new_treat['ImagesTaken_total'] = data_part2.groupby(['PatientSerNum', 'date']).ImagesTaken.sum()

print('Start MU_total')
new_treat['MU_total'] = data_part2.groupby(['PatientSerNum', 'date']).MU.sum()

print('Start MUCoeff_total')
new_treat['MUCoeff_total'] = data_part2.groupby(['PatientSerNum', 'date']).MUCoeff.sum()

print('Start TreatmentTime_total')
new_treat['TreatmentTime_total'] = data_part2.groupby(['PatientSerNum', 'date']).TreatmentTime.sum()

Start categorical features

Start FractionNumber
Start UserName
Start RadiationSerNum
Start RadiationId
Start ResourceSerNum
Start CourseId
Start PatientSerNum
Start date
Start numberical features

Start ImagesTaken_total
Start MU_total
Start MUCoeff_total
Start TreatmentTime_total


In [52]:
new_treat.drop('ResourceSerNum', axis = 1, inplace = True)

In [56]:
new_appt.reset_index(drop = True, inplace = True)
new_treat.reset_index(drop = True, inplace = True)

print(f'The shape of new_appt is {new_appt.shape}')
print(f'The shape of new_treat is {new_treat.shape}')

DATA = pd.merge(new_appt, new_treat, on = ['PatientSerNum', 'date'], how = 'inner')
DATA.sort_values(by = ['PatientSerNum', 'AppointmentSerNum', 'ScheduledStartTime'], inplace = True)
print(f'The shape of data is {DATA.shape}')

The shape of new_appt is (321034, 20)
The shape of new_treat is (142974, 11)
The shape of data is (139944, 29)


In [57]:
DATA = pd.merge(DATA, data_re_appt[['AppointmentSerNum', 'ResourceSerNum']], on = 'AppointmentSerNum', how = 'inner')
print(f'The shape of data is {DATA.shape}')

The shape of data is (158312, 30)


In [58]:
print(f'The shape of data is {DATA.shape}')
DATA_ = DATA[(DATA.Actual_duration >= 10) &
             (DATA.Actual_duration <= 60)]
print(f'The shape of data is {DATA_.shape}')

The shape of data is (158312, 30)
The shape of data is (143900, 30)


=========================================================

In [5]:
sql_re = """SELECT * FROM resource"""
data_re_ = ml_df.get_df_from_sql(sql_re)
print(f'The shape of data_resource {data_re_.shape}')
sql_re_appt = """SELECT * FROM resourceappointment"""
data_re_appt_ = ml_df.get_df_from_sql(sql_re_appt)
print(f'The shape of data_re_appt {data_re_appt_.shape}')
sql_appt = """SELECT * FROM appointment"""
data_appt_ = ml_df.get_df_from_sql(sql_appt)
print(f'The shape of data_appt {data_appt_.shape}')
sql_pl = """SELECT * FROM patientlocation"""
data_pl_ = ml_df.get_df_from_sql(sql_pl)
print(f'The shape of data_pl {data_pl_.shape}')
sql_plmh = """SELECT * FROM patientlocationmh"""
data_plmh_ = ml_df.get_df_from_sql(sql_plmh)
print(f'The shape of data_plmh {data_plmh_.shape}')

The shape of data_resource (477, 6)
The shape of data_re_appt (475920, 7)
The shape of data_appt (1074478, 17)
The shape of data_pl (206855, 9)
The shape of data_plmh (2333039, 10)


In [6]:
try:
    print('Drop columns')
    data_appt_.drop(columns = ['LastUpdated'], inplace = True)
    data_re_.drop(columns = ['LastUpdated'], inplace = True)
    data_re_appt_.drop('LastUpdated', axis = 1, inplace = True)
    data_pl_.drop(columns = ['LastUpdated'], inplace = True)
    data_plmh_.drop(columns = ['LastUpdated'], inplace = True)
except:
    print('Finish droppping columns')

Drop columns


In [7]:
data_re = data_re_.copy()
data_re_appt = data_re_appt_.copy()
data_appt = data_appt_.copy()
data_pl = data_pl_.copy()
data_plmh = data_plmh_.copy()

In [8]:
print(f'The shape of data_appt {data_appt.shape}')
data_appt = data_appt[
    (data_appt.ActualStartDate != datetime.datetime(1970, 1, 1, 0, 0)) &
    (data_appt.ActualEndDate != datetime.datetime(1970, 1, 1, 0, 0)) &
    (data_appt.ActualStartDate != '0000-00-00 00:00:00')]
print(f'The shape of data_appt {data_appt.shape}')

# 已经检查过，一个Appointment 里面只有一个AliasSerNum
# 一个AliasSerNum 对应多个ResourceSerNum
# 一个ResourceSerNum 对应多个AppointmentSerNum
print('Merge data_appt and data_re_appt')
appt_re_appt = pd.merge(data_appt, data_re_appt, on = ['AppointmentSerNum', 'AliasSerNum'], how = 'inner')
print(f'The shape of appt_re_appt {appt_re_appt.shape}')
appt_re_appt.drop('AliasSerNum', axis = 1, inplace = True)

print('\nMerge data_re')
appt_loc = pd.merge(data_re, appt_re_appt, on = ['ResourceSerNum'], how = 'inner')
print(f'The shape of appt_loc {appt_loc.shape}')

print('\nMerge data_pl')
data_pl.drop('AliasSerNum', axis = 1, inplace = True)
appt_loc_pl = pd.merge(data_pl, appt_loc, on = ['AppointmentSerNum'], how = 'right')
print(f'The shape of appt_loc_pl {appt_loc_pl.shape}')

print('\nGet Treat_Date')
appt_loc_pl['Treat_Date'] = appt_loc_pl.apply(lambda x: x.ActualStartDate.strftime("%Y-%m-%d"), axis = 1)

appt_loc_pl = appt_loc_pl[[
    'AppointmentSerNum', 'PatientSerNum', 'AliasExpressionSerNum',
    'ScheduledStartTime', 'ScheduledEndTime',
    'ActualStartDate', 'ActualEndDate',
    'ResourceSerNum']]
print(f'The shape of room_infomation {appt_loc_pl.shape}')
appt_loc_pl.head()

The shape of data_appt (1074478, 16)
The shape of data_appt (370299, 16)
Merge data_appt and data_re_appt
The shape of appt_re_appt (165512, 20)

Merge data_re
The shape of appt_loc (165512, 23)

Merge data_pl
The shape of appt_loc_pl (165512, 29)

Get Treat_Date
The shape of room_infomation (165512, 8)


,AppointmentSerNum,PatientSerNum,AliasExpressionSerNum,ScheduledStartTime,ScheduledEndTime,ActualStartDate,ActualEndDate,ResourceSerNum
0,849926,38996,1505,2015-10-19 14:00:00,2015-10-19 14:15:00,2015-10-19 14:18:00,2015-10-19 14:34:00,176
1,849927,38996,1505,2015-10-20 13:45:00,2015-10-20 14:00:00,2015-10-20 13:32:00,2015-10-20 13:48:00,177
2,849928,38996,1505,2015-10-21 15:30:00,2015-10-21 15:45:00,2015-10-21 14:57:00,2015-10-21 15:07:00,176
3,849929,38996,1505,2015-10-22 14:00:00,2015-10-22 14:15:00,2015-10-22 13:51:00,2015-10-22 13:59:00,176
4,849930,38996,1505,2015-10-23 13:00:00,2015-10-23 13:15:00,2015-10-23 12:52:00,2015-10-23 13:01:00,176


In [59]:
appt_loc_pl = appt_loc_pl[[
    'AppointmentSerNum', 'PatientSerNum', 'AliasExpressionSerNum', 'ScheduledStartTime', 'ResourceSerNum']]
# appt_loc_pl['ScheduledStartTime'] = appt_loc_pl['ScheduledStartTime'].apply(lambda x: Datetime_to_str(x))

room_info = pd.merge(appt_loc_pl, DATA_, 
                     on = ['PatientSerNum', 'AppointmentSerNum', 'ScheduledStartTime', 'ResourceSerNum'], how = 'inner')
print(f'The shape of room_infomation {room_info.shape}')

The shape of room_infomation (143964, 31)


In [60]:
room_info.columns

Index(['AppointmentSerNum', 'PatientSerNum', 'AliasExpressionSerNum',
       'ScheduledStartTime', 'ResourceSerNum', 'Sex', 'DoctorSerNum', 'date',
       'ScheduledEndTime', 'ActualStartDate', 'ActualEndDate', 'dxt_AliasName',
       'AliasSerNum', 'CourseSerNum', 'PlanSerNum', 'TreatmentOrientation',
       'month', 'week', 'hour', 'age', 'Scheduled_duration', 'Actual_duration',
       'FractionNumber', 'UserName', 'RadiationSerNum', 'RadiationId',
       'CourseId', 'ImagesTaken_total', 'MU_total', 'MUCoeff_total',
       'TreatmentTime_total'],
      dtype='object')

In [61]:
room_info_grouped = room_info.groupby(['ResourceSerNum', 'date'])
room_info_grouped.groups.keys()

dict_keys([145, 175, 176, 177, 178, 179, 180, 237, 281, 293, 301, 303, 304, 310, 359, 363, 365, 366, 369, 370, 371, 373, 391, 406, 416, 437, 666, 672, 697])

In [62]:
sample = room_info_grouped.get_group(175)
sample.shape

(20563, 31)

In [65]:
sample.columns

Index(['AppointmentSerNum', 'PatientSerNum', 'AliasExpressionSerNum',
       'ScheduledStartTime', 'ResourceSerNum', 'Sex', 'DoctorSerNum', 'date',
       'ScheduledEndTime', 'ActualStartDate', 'ActualEndDate', 'dxt_AliasName',
       'AliasSerNum', 'CourseSerNum', 'PlanSerNum', 'TreatmentOrientation',
       'month', 'week', 'hour', 'age', 'Scheduled_duration', 'Actual_duration',
       'FractionNumber', 'UserName', 'RadiationSerNum', 'RadiationId',
       'CourseId', 'ImagesTaken_total', 'MU_total', 'MUCoeff_total',
       'TreatmentTime_total'],
      dtype='object')

In [76]:
sample_grouped.groups.keys()

dict_keys(['2015--06--15', '2015--06--16', '2015--06--17', '2015--06--18', '2015--06--19', '2015--06--22', '2015--06--23', '2015--06--25', '2015--06--26', '2015--06--29', '2015--06--30', '2015--07--02', '2015--07--03', '2015--07--06', '2015--07--07', '2015--07--08', '2015--07--09', '2015--07--10', '2015--07--13', '2015--07--14', '2015--07--15', '2015--07--16', '2015--07--17', '2015--07--20', '2015--07--21', '2015--07--22', '2015--07--23', '2015--07--24', '2015--07--27', '2015--07--28', '2015--07--29', '2015--07--30', '2015--07--31', '2015--08--01', '2015--08--04', '2015--08--05', '2015--08--06', '2015--08--07', '2015--08--10', '2015--08--11', '2015--08--12', '2015--08--13', '2015--08--14', '2015--08--17', '2015--08--18', '2015--08--19', '2015--08--20', '2015--08--21', '2015--08--24', '2015--08--25', '2015--08--26', '2015--08--27', '2015--08--28', '2015--08--31', '2015--09--01', '2015--09--02', '2015--09--03', '2015--09--04', '2015--09--08', '2015--09--09', '2015--09--10', '2015--09--11

In [112]:
sample_grouped = sample.groupby('date')
room = sample_grouped.get_group('2017--11--09')
room.sort_values(by = 'ActualStartDate', inplace = True)

room['pre_ActualEndDate'] = room.ActualEndDate.shift(periods = 1, fill_value = room.ActualStartDate.iloc[0])
room['Transfer_duration'] = room.apply(lambda x: cal_time_inv(x.pre_ActualEndDate,x.ActualStartDate), axis = 1)
room['Interval_number'] = room.apply(lambda x: int(x.Transfer_duration / 60 / 15), axis = 1)

room.Transfer_duration.iloc[-1] = cal_time_inv(room.ActualEndDate.iloc[-2], room.ScheduledStartTime.iloc[-1])
room.Interval_number.iloc[-1] = int(room.Transfer_duration.iloc[-1] /60 / 15)

In [113]:
room[['ScheduledStartTime', 'ScheduledEndTime', 'pre_ActualEndDate', 'ActualStartDate', 'ActualEndDate', 'Transfer_duration', 'Interval_number']]

,ScheduledStartTime,ScheduledEndTime,pre_ActualEndDate,ActualStartDate,ActualEndDate,Transfer_duration,Interval_number
65246,2017-11-09 08:45:00,2017-11-09 09:00:00,2017-11-09 07:40:00,2017-11-09 07:40:00,2017-11-09 07:57:00,-1,0
65270,2017-11-09 09:00:00,2017-11-09 09:15:00,2017-11-09 07:57:00,2017-11-09 08:54:00,2017-11-09 09:20:00,3420,3
65274,2017-11-09 09:30:00,2017-11-09 09:45:00,2017-11-09 09:20:00,2017-11-09 09:20:00,2017-11-09 09:33:00,-1,0
65282,2017-11-09 09:15:00,2017-11-09 09:30:00,2017-11-09 09:33:00,2017-11-09 09:44:00,2017-11-09 09:54:00,660,0
65284,2017-11-09 10:30:00,2017-11-09 10:45:00,2017-11-09 09:54:00,2017-11-09 09:57:00,2017-11-09 10:12:00,180,0
65279,2017-11-09 10:15:00,2017-11-09 10:30:00,2017-11-09 10:12:00,2017-11-09 10:13:00,2017-11-09 10:25:00,60,0
65271,2017-11-09 10:00:00,2017-11-09 10:15:00,2017-11-09 10:25:00,2017-11-09 10:25:00,2017-11-09 10:37:00,-1,0
65297,2017-11-09 11:00:00,2017-11-09 11:15:00,2017-11-09 10:37:00,2017-11-09 10:38:00,2017-11-09 10:48:00,60,0
65299,2017-11-09 10:45:00,2017-11-09 11:00:00,2017-11-09 10:48:00,2017-11-09 10:50:00,2017-11-09 11:15:00,120,0
65316,2017-11-09 11:30:00,2017-11-09 11:45:00,2017-11-09 11:15:00,2017-11-09 11:18:00,2017-11-09 11:43:00,180,0


In [ ]:
sample_grouped.get_group('2015-05-29')[[
    'AppointmentSerNum', 'PatientSerNum', 'AppointmentAriaSer',
    'ActivityInstanceAriaSer', 'DiagnosisSerNum', 'PrioritySerNum',
    'AliasExpressionSerNum', 'ScheduledStartTime', 'ScheduledEndTime',
    'ActualStartDate', 'ActualEndDate', 'ResourceAppointmentSerNum']]